# Detecção de Objetos

Este notebook introduz os conceitos fundamentais da detecção de objetos, uma tarefa de visão computacional que envolve identificar e localizar múltiplos objetos dentro de uma imagem. Abordaremos uma implementação inspirada na arquitetura YOLO (You Only Look Once), focando na criação de um modelo capaz de prever caixas delimitadoras (bounding boxes) e classes para objetos. Para simplificar o problema e focar nos mecanismos centrais, construiremos um conjunto de dados sintético usando dígitos do MNIST, onde cada imagem conterá três dígitos em posições aleatórias.

In [ ]:
import math
import random
import numpy as np
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Geração do Dataset Sintético

Para treinar um modelo de detecção de objetos, precisamos de um conjunto de dados que forneça não apenas as imagens, mas também as caixas delimitadoras (bounding boxes) e as classes para cada objeto. A classe `ObjectsMNIST` implementa um `Dataset` customizado do PyTorch para gerar dados sintéticos em tempo real.

Esta abordagem é mais robusta que a anterior:
1.  Utiliza a biblioteca `PIL` (Pillow) para criar a tela (`canvas`) e colar os dígitos.
2.  Aplica transformações de data augmentation mais realistas em cada dígito antes de colá-lo: uma leve variação de escala (`random.uniform(0.9, 1.3)`) e uma rotação (`random.uniform(-10, 10)`).
3.  Implementa um mecanismo de anticolisão.

In [ ]:
class ObjectsMNIST(Dataset):
    def __init__(self, root="./data", split="train", n_samples=5000, img_size=96, n_digits=3, iou_thr=0.15, max_tries=50):
        self.n_samples, self.img_size, self.n_digits = n_samples, img_size, n_digits
        self.iou_thr, self.max_tries = iou_thr, max_tries
        base = datasets.MNIST(root=root, train=(split=="train"), download=True)
        self.images, self.labels = base.data.numpy(), base.targets.numpy()

    def __len__(self):
        return self.n_samples

    def _iou(self, a, b):
        # a,b = (cx,cy,w,h) normalizados
        xa1, ya1, xa2, ya2 = a[0]-a[2]/2, a[1]-a[3]/2, a[0]+a[2]/2, a[1]+a[3]/2
        xb1, yb1, xb2, yb2 = b[0]-b[2]/2, b[1]-b[3]/2, b[0]+b[2]/2, b[1]+b[3]/2
        inter = max(0, min(xa2, xb2)-max(xa1, xb1)) * max(0, min(ya2, yb2)-max(ya1, yb1))
        area_a, area_b = (xa2-xa1)*(ya2-ya1), (xb2-xb1)*(yb2-yb1)
        return inter / (area_a + area_b - inter + 1e-9)

    def __getitem__(self, idx):
        canvas = Image.new("L", (self.img_size, self.img_size), 0)
        ids = np.random.choice(len(self.images), self.n_digits, replace=False)
        targets, boxes = [], []

        for i in ids:
            # pequena escala e rotação
            img = Image.fromarray(self.images[i])
            s = int(28 * random.uniform(0.9, 1.3))
            img = img.resize((s, s), Image.BILINEAR)
            if random.random() < 0.4:
                img = img.rotate(random.uniform(-10, 10), expand=True, fillcolor=0)

            # tenta colocar sem sobreposição
            for _ in range(self.max_tries):
                x, y = random.randint(0, self.img_size - img.width), random.randint(0, self.img_size - img.height)
                box = ((x + img.width/2)/self.img_size, (y + img.height/2)/self.img_size,
                       img.width/self.img_size, img.height/self.img_size)
                if all(self._iou(box, b) < self.iou_thr for b in boxes):
                    canvas.paste(img, (x, y))
                    boxes.append(box)
                    targets.append((int(self.labels[i]), *box))
                    break

        x = transforms.ToTensor()(canvas)
        y = torch.tensor(targets, dtype=torch.float32)
        return x, y

In [ ]:
# Data
IMG_SIZE = 128
BATCH_SIZE = 64

train_dataset = ObjectsMNIST(split="train", n_samples=10000, img_size=IMG_SIZE, n_digits=3)
test_dataset  = ObjectsMNIST(split="test", n_samples=1000, img_size=IMG_SIZE, n_digits=3)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
def plot_dataset_examples(dataset, n=8, cols=4, figsize_scale=3.0):
    assert n > 0 and cols > 0
    rows = int(np.ceil(n / cols))
    img_size = getattr(dataset, "img_size", None)
    
    # Seleciona índices aleatórios sem criar DataLoader
    idxs = random.sample(range(len(dataset)), k=n if n < len(dataset) else len(dataset))
    plt.figure(figsize=(figsize_scale*cols, figsize_scale*rows))

    for p, idx in enumerate(idxs, start=1):
        img_t, targets = dataset[idx]                     # img_t: [1,H,W], targets: [K,5]
        img_np = img_t.squeeze(0).numpy()                 # [H,W] para imshow
        H, W = img_np.shape
        if img_size is None:  # se não existir atributo, infere da imagem
            img_size = max(H, W)

        ax = plt.subplot(rows, cols, p)
        ax.imshow(img_np, cmap="gray")
        ax.axis("off")

        # Desenha as caixas ground truth
        for row in targets.tolist():
            cls, cx, cy, w, h = row
            # converte para pixels
            x1 = (cx - w/2) * W
            y1 = (cy - h/2) * H
            ww = w * W
            hh = h * H
            rect = patches.Rectangle((x1, y1), ww, hh, linewidth=1.5, edgecolor='g', facecolor='none')
            ax.add_patch(rect)
            ax.text(x1, y1 - 2, f"{int(cls)}", color='g', fontsize=8, va="bottom")

    plt.tight_layout()
    plt.show()

plot_dataset_examples(train_dataset, n=8, cols=4)

## Arquitetura do Modelo

Esta célula define uma arquitetura de rede neural convolucional (CNN) totalmente convolucional, inspirada no design do YOLO. A arquitetura é construída usando uma função auxiliar `block(cin, cout)` que atua como a unidade fundamental de downsampling. Este bloco contém uma `nn.Conv2d` com `stride=2` (que reduz a dimensão espacial pela metade), seguida por `nn.BatchNorm2d` para estabilização e uma ativação `nn.ReLU`.

O `self.backbone` é um `nn.Sequential` que empilha quatro desses blocos, reduzindo a dimensão espacial da entrada (ex: 96x96) sequencialmente (96 -> 48 -> 24 -> 12 -> 6). A saída do backbone será um mapa de características com `S=6`. O `self.head` é uma única camada `nn.Conv2d` com kernel `1x1` que atua como o preditor, mapeando os canais de características (128) para o tensor de predição final com `1+4+C` canais, correspondendo ao score de objectness (1), às coordenadas da caixa (4) e aos logits de classe (C). O método `forward` conclui permutando a saída para o formato `[B, S, S, 1+4+C]`, facilitando o cálculo da perda.

In [ ]:
class MiniYOLO(nn.Module):
    def __init__(self, S, C):
        super().__init__()
        self.S, self.C = S, C
        def block(cin, cout): 
            return nn.Sequential(
                nn.Conv2d(cin, cout, 3, 2, 1, bias=False),
                nn.BatchNorm2d(cout),
                nn.ReLU(inplace=True)
            )
        self.backbone = nn.Sequential(
            block(1, 32),   # 128 -> 64
            block(32, 64),  # 64 -> 32
            block(64, 128), # 32 -> 16
            block(128, 128) # 16 -> 8
        )
        self.head = nn.Conv2d(128, 1+4+C, 1)

    def forward(self, x):
        f = self.backbone(x)                 # [B,128,S,S]
        y = self.head(f).permute(0,2,3,1)    # [B,S,S,1+4+C]
        return y

## Função de Perda

Implementamos uma `nn.Module` customizada para a função de perda, pois ela é composta por múltiplos termos tratados de forma diferente. Esta implementação eficiente da perda YOLO é a soma ponderada de três componentes.

O primeiro é a **Perda de Confiança (Objectness Loss)**, que utiliza `BCEWithLogitsLoss` para comparar a predição de objectness (`obj_p`) com o alvo (`obj_t`). Esta perda é calculada sobre *todas* as células da grade, penalizando tanto falsos positivos quanto falsos negativos.

Os outros dois componentes são aplicados seletivamente. A **Perda de Localização (Box Loss)** usa `L1Loss` para regredir as coordenadas da caixa. A **Perda de Classificação (Class Loss)** usa `CrossEntropyLoss` para a classe. Crucialmente, ambas as perdas (box e class) são calculadas *apenas* nas células onde um objeto está realmente presente. Isso é alcançado através de uma `mask` booleana (`obj_t > 0.5`) que filtra apenas as predições e alvos relevantes.

A perda final é `obj_loss + self.lb*box_loss + self.lc*cls_loss`, onde `lb` e `lc` ponderam a importância relativa da localização e classificação.

In [ ]:
class YoloLoss(nn.Module):
    def __init__(self, lambda_box=5.0, lambda_cls=1.0):
        super().__init__()
        self.lb, self.lc = lambda_box, lambda_cls
        self.bce = nn.BCEWithLogitsLoss(reduction="none")
        self.ce  = nn.CrossEntropyLoss(reduction="none")

    def forward(self, pred, target):
        obj_t = target[..., 0]         # [B,S,S]
        box_t = target[..., 1:5]       # [B,S,S,4]
        cls_t = torch.argmax(target[..., 5:], dim=-1)  # [B,S,S]

        obj_p = pred[..., 0]
        box_p = pred[..., 1:5]
        cls_p = pred[..., 5:]

        obj_loss = self.bce(obj_p, obj_t).mean()

        mask = (obj_t > 0.5).unsqueeze(-1)  # [B,S,S,1]
        if mask.any():
            box_loss = F.l1_loss(box_p[mask.expand_as(box_p)], box_t[mask.expand_as(box_t)], reduction="mean")
            cls_loss = self.ce(cls_p[obj_t>0.5], cls_t[obj_t>0.5]).mean()
        else:
            box_loss = torch.tensor(0., device=pred.device)
            cls_loss = torch.tensor(0., device=pred.device)

        return obj_loss + self.lb*box_loss + self.lc*cls_loss

In [ ]:
# Modelo
S = 8
C = 10
model = MiniYOLO(S=S, C=C).to(device)

In [ ]:
# Loss e otimizador
criterion = YoloLoss(lambda_box=5.0, lambda_cls=1.0)
optim = torch.optim.Adam(model.parameters(), lr=2e-3)

## Construção dos Alvos

O modelo produz uma grade `[B, S, S, 1+4+C]`, mas o `Dataset` fornece uma lista de tensores `[K_i, 5]`. A função `build_targets` é uma etapa de pré-processamento essencial que converte o formato do "ground-truth" do dataset para o formato de grade exigido pela função de perda.

Para cada imagem no lote, a função itera sobre seus objetos "ground-truth" e determina a célula da grade `(i, j)` responsável por cada objeto, com base no seu centro `(cx, cy)`. Um mecanismo de **Tratamento de Colisão** é implementado: se múltiplos objetos forem mapeados para a mesma célula `(i, j)`, a estratégia é manter apenas o objeto com a maior área (`w*h`).

Além disso, a função aplica uma **Transformação de Coordenadas**, uma técnica comum em implementações YOLO. O alvo não armazena `(cx, cy, w, h)` diretamente, mas sim valores transformados que são mais fáceis para a rede regredir. As coordenadas `tx, ty` são calculadas como a posição do centro relativa ao canto da célula (`cx*S - i`), e as dimensões `tw, th` são armazenadas no espaço logarítmico (`math.log(w*S)`). O tensor `T` na posição `[b, j, i]` é então preenchido com o vetor final: `[1.0 (objectness), tx, ty, tw, th, ...one-hot class...]`.

In [ ]:
def build_targets(batch_targets, S, C):
    B = len(batch_targets)
    T = torch.zeros(B, S, S, 1+4+C)

    for b, targets in enumerate(batch_targets):
        if targets.numel() == 0:
            continue

        # Mantém a melhor box por célula (maior área)
        best = {}

        for cls, cx, cy, w, h in targets:
            i = int((cx * S).clamp(0, S-1).floor())
            j = int((cy * S).clamp(0, S-1).floor())
            area = w*h

            if (j,i) in best and area <= best[(j,i)][0]:
                continue

            tx, ty = cx*S - i, cy*S - j
            tw, th = torch.log(w*S + 1e-6), torch.log(h*S + 1e-6)

            vec = torch.zeros(1+4+C)
            vec[0] = 1
            vec[1:5] = torch.tensor([tx, ty, tw, th])
            vec[5 + int(cls)] = 1

            best[(j,i)] = (area, vec)

        for (j,i), (_, vec) in best.items():
            T[b, j, i] = vec

    return T

## Loop de Treinamento

O treinamento é encapsulado na função `run_epoch`, que executa uma época completa de treinamento ou validação. A função recebe um `loader` e um booleano `train` que alterna o modelo entre os modos `train` e `eval` usando `model.train(mode=train)`.

Dentro do loop do lote, a etapa crítica é a chamada `T = build_targets(...)`, que converte os alvos `t` (vindos do `DataLoader`) para o tensor de grade `T` esperado pela função de perda (`criterion`). O cálculo do `forward pass` e da perda é envolvido em um contexto `torch.set_grad_enabled(train)`, que habilita o cálculo de gradientes apenas durante o treinamento. Se `train` for verdadeiro, o `backward pass` e a atualização do otimizador são executados. O loop principal itera pelo número de `EPOCHS`, chamando `run_epoch` para os dados de treino e teste e imprimindo as perdas resultantes.

In [ ]:
# Treinamento
EPOCHS = 4

def run_epoch(loader, train=True):
    model.train(mode=train)
    total = 0.0
    for x, t in loader:
        x = x.to(device)
        T = build_targets([y for y in t], S, C).to(device)
        with torch.set_grad_enabled(train):
            y = model(x)
            loss = criterion(y, T)
            if train:
                optim.zero_grad(); loss.backward(); optim.step()
        total += loss.item() * x.size(0)
    return total / len(loader.dataset)

for ep in range(1, EPOCHS+1):
    tr = run_epoch(train_loader, train=True)
    va = run_epoch(test_loader, train=False)
    print(f"epoch {ep:02d} | train {tr:.4f} | val {va:.4f}")

In [ ]:
# Salva o modelo treinado
torch.save(model.state_dict(), "miniyolo_mnist.pth")

In [ ]:
# Carrega o modelo treinado
model = MiniYOLO(S=S, C=C).to(device)
model.load_state_dict(torch.load("miniyolo_mnist.pth"))

## Inferência e Visualização dos Resultados

Para avaliar visualmente o modelo, precisamos decodificar o tensor de saída bruto da rede `[S, S, 1+4+C]` em uma lista de caixas delimitadoras legíveis.

A função `decode_boxes` é responsável por essa decodificação, revertendo o processo de `build_targets`. Ela itera por cada célula `(j, i)` da grade, aplica `sigmoid` na confiança e `softmax` nas classes, e filtra as células com confiança abaixo de um limiar `conf`. Para as células retidas, ela reverte as transformações de coordenadas: `tx` é convertido de volta para `cx` (coordenada relativa à imagem) usando `(i + torch.sigmoid(tx)) / S`, e `tw` é convertido de volta para `w` usando `torch.exp(tw) / S`. O score final é calculado como `P(Object) * P(Class | Object)`.

A função `show_samples` utiliza a `decode_boxes` para a visualização. Ela obtém um lote de teste, executa o modelo e, para cada imagem, desenha tanto as caixas "ground-truth" (em verde) quanto as caixas previstas pela `decode_boxes` (em vermelho), anotadas com a classe e o score.

In [ ]:
def decode_boxes(cell_pred, S, conf=0.4):
    obj = torch.sigmoid(cell_pred[..., 0])
    box = cell_pred[..., 1:5]
    cls = F.softmax(cell_pred[..., 5:], dim=-1)
    out = []
    for j in range(S):
        for i in range(S):
            if obj[j,i] < conf: 
                continue
            tx, ty, tw, th = box[j,i]
            cx = (i + torch.sigmoid(tx)) / S
            cy = (j + torch.sigmoid(ty)) / S
            w  = torch.exp(tw) / S
            h  = torch.exp(th) / S
            c  = int(torch.argmax(cls[j,i]).item())
            s  = float(obj[j,i] * cls[j,i,c])
            out.append((s, c, float(cx), float(cy), float(w), float(h)))
    return out

In [ ]:
def show_samples(n=6, conf=0.5):
    model.eval()
    with torch.no_grad():
        x, t = next(iter(test_loader))
        x = x.to(device)
        y = model(x).cpu()
        imgs = x.cpu().numpy()
        k = min(n, x.size(0))

    plt.figure(figsize=(3*k, 3))
    for idx in range(k):
        ax = plt.subplot(1, k, idx+1)
        ax.imshow(imgs[idx,0], cmap="gray"); ax.axis("off")
        # GT
        for lab, cx, cy, w, h in t[idx].tolist():
            x1, y1 = (cx-w/2)*IMG_SIZE, (cy-h/2)*IMG_SIZE
            rect = plt.Rectangle((x1,y1), w*IMG_SIZE, h*IMG_SIZE, fill=False, ec="g", lw=1.5)
            ax.add_patch(rect); ax.text(x1, y1-2, f"gt:{int(lab)}", color="g", fontsize=8)
        # Pred
        boxes = decode_boxes(y[idx], S=S, conf=conf)
        for sc, cl, cx, cy, w, h in boxes:
            x1, y1 = (cx-w/2)*IMG_SIZE, (cy-h/2)*IMG_SIZE
            rect = plt.Rectangle((x1,y1), w*IMG_SIZE, h*IMG_SIZE, fill=False, ec="r", lw=1.5)
            ax.add_patch(rect); ax.text(x1, y1-2, f"p:{cl}@{sc:.2f}", color="r", fontsize=8)
    plt.tight_layout(); plt.show()

show_samples(n=4, conf=0.5)